In [1]:
import pyvisa
# import qcodes as qc
# import Aalto_NANOdrivers as aa
import matplotlib.pyplot as plt
import numpy as np
from numpy import *

In [3]:
from qcodes_contrib_drivers.drivers.StanfordResearchSystems import SIM928

# Scan through all GPIB channels for instruments

In [2]:
rm = pyvisa.ResourceManager()
list_of_instrument = rm.list_resources()
for i in range(len(list_of_instrument)):
    my_instrument = rm.open_resource(list_of_instrument[i])
    try:
        listen = my_instrument.query('*IDN?')
        print('Found instrument at', i, list_of_instrument[i], '-> ', listen[:-2])
    except: pass

Found instrument at 1 GPIB0::23::INSTR ->  Rohde-Schwarz,ZNB8-2Port,1311601042102249,2.8


In [4]:
list_of_instrument[15]

'GPIB0::7::INSTR'

# Mannual initialization of any instrument

In [5]:
my_instrument = rm.open_resource(list_of_instrument[15])
my_instrument.query('*IDN?')

'ANRITSU,MS2830A,6200847105,4.09.01\n'

# Stanford Research Systems SIM900.SIM928 via driver

In [6]:
"""
if you will run this sectionn twice, driver will try to connect to the device with the same name SIM928 
and then break since the name is already used.

Modules are single SIM928 devises connected to a mainframe SIM900

https://qcodes.github.io/Qcodes_contrib_drivers/api/generated/qcodes_contrib_drivers.drivers.StanfordResearchSystems.html

""" 

VoltGen = SIM928.SIM928(name='SIM928', address='GPIB1::16::INSTR') 
modules = VoltGen.find_modules() 
modules

[SIM928(SIM928)] Could not connect at GPIB1::16::INSTR
Traceback (most recent call last):
  File "C:\Users\Demag\anaconda3\lib\site-packages\qcodes\instrument\visa.py", line 133, in _connect_and_handle_error
    visa_handle, visabackend = self._open_resource(address, visalib)
  File "C:\Users\Demag\anaconda3\lib\site-packages\qcodes\instrument\visa.py", line 160, in _open_resource
    resource = resource_manager.open_resource(address)
  File "C:\Users\Demag\anaconda3\lib\site-packages\pyvisa\highlevel.py", line 3247, in open_resource
    info = self.resource_info(resource_name, extended=True)
  File "C:\Users\Demag\anaconda3\lib\site-packages\pyvisa\highlevel.py", line 3174, in resource_info
    ret, err = self.visalib.parse_resource_extended(self.session, resource_name)
  File "C:\Users\Demag\anaconda3\lib\site-packages\pyvisa\ctwrapper\functions.py", line 2077, in parse_resource_extended
    ret = library.viParseRsrcEx(
  File "C:\Users\Demag\anaconda3\lib\site-packages\pyvisa\ctwrap

VisaIOError: VI_ERROR_INTF_NUM_NCONFIG (-1073807195): The interface type is valid but the specified interface number is not configured.

In [ ]:
"""
Use this section to set voltage. Use module number as first argument. Command "set_voltage" doesn't return anything

""" 

listen = VoltGen.set_voltage(1, 0.0)
listen

In [ ]:
"""
Use this section to read voltage. Use module number as first argument. Command "get_voltage" returns voltage in ??volts??

""" 
listen = VoltGen.get_voltage(1)
listen

# Rohde-Schwarz,ZNB20

In [ ]:
from qcodes.instrument_drivers.rohde_schwarz import (
    RohdeSchwarzZNB8,
    RohdeSchwarzZNBChannel,
)

In [ ]:
vna = RohdeSchwarzZNB8('VNA', 'GPIB0::20::INSTR')
vna.add_channel('S21')
vna.display_single_window()

In [ ]:
vna.channels.S21.power(-30)
vna.channels.autoscale()
vna.channels.S21.start(6e9)
vna.channels.S21.stop(6.8e9)
vna.channels.S21.npts(1000)

In [ ]:
freq_range = np.array([5e9, 8e9])
nsteps = 5000
fr0 = np.linspace(freq_range[0], freq_range[1], nsteps)
vna.rf_on()
vna.channels.S21.start(freq_range[0])
vna.channels.S21.stop(freq_range[1])
vna.channels.S21.npts(nsteps)
get_v = vna.channels.S21.trace_mag_phase.get()
vna.rf_off()

fig, ax = plt.subplots()
ax.plot(fr0, get_v[0])

# ANAPICO 

In [ ]:
LAN_ADRESS = 'TCPIP0::169.254.5.91::inst0::INSTR' #floating. use NI MAX to find current adress

In [ ]:
import pyvisa
import time

rm = pyvisa.ResourceManager()
anapico = rm.open_resource(LAN_ADRESS)
anapico.query('*IDN?')

In [ ]:
anapico.query(r'*SOUR {}: POW {}'.format(str(1), str(-30)))

In [ ]:
anapico.write(r'SOUR1:POW -30')
anapico.write(r'OUTP1 OFF')
anapico.write(r'SOUR1:FREQ 2e6')

In [ ]:
anapico.query(r'SOUR1:FREQ?')

In [ ]:
anapico.write(r'OUTP1 ON')